# Importing the necessary libraries, in our case we are using pytorch


In [1]:
# Importing necessary libraries
import torch  # PyTorch library for deep learning
from torch import nn, optim  # Neural network modules and optimization algorithms
from torch.utils.data import Dataset, DataLoader, random_split  # Tools for handling datasets
from torchvision import transforms, models  # Vision-related utilities including pre-trained models
from matplotlib import pyplot as plt  # Plotting library for visualization
from PIL import Image  # Python Imaging Library for image processing
from tqdm import tqdm  # Progress bar for tracking iterations
import os  # Operating system utilities


# Be sure to change the directory of images, and look at instructions below on how to run the model without retraining the weights


In [2]:
# Configuration Variables

# Path to the folder containing images
data_folder = './images'

# Number of classes in the dataset
classes = 5

# Learning rate for the optimizer
lr = 0.001

# Batch size for training
batch_size = 32

# Number of epochs for training
epochs = 40

# Size of the input images (assumed to be square)
image_size = 224

# Device selection based on GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Loading the VGG16 model with default weights
model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)


In [3]:
def freeze(model):
    # Function to freeze the parameters of a given model
    for param in model.parameters():
        param.requires_grad = False

# Getting the number of input features for the last layer of the model's classifier
num_features = model.classifier[-1].in_features

# Flag to determine if we want to freeze the features of the model
freeze_features = True

# If freeze_features flag is set to True, freeze the parameters of the model
if freeze_features:
    freeze(model)

# Updating the last layer of the model's classifier to output classes specific to our problem
model.classifier[-1] = nn.Linear(num_features, classes)


In [4]:
#WARNING: USE THIS AT YOUR OWN RISK!!!!!!!!!!!!!!!!!!!!!!!!
def visualize(image, label):
  image = torch.permute(image, (2, 1, 0)).numpy()
  plt.figure()
  plt.imshow(image)
  plt.title(str(label))
  plt.xticks([])
  plt.yticks([])

In [5]:
class CreateDataset(Dataset):
  def __init__(self, source_dir, transform = None):

    #Variables to keep track of
    self.labelDictionary = {}
    self.images = []
    self.labels = []
    self.transform = transform
    self.toTensor = transforms.Compose([
      transforms.ToTensor(),
    ])

    #If the source directory provided isn't a directory, return -1
    if os.path.isdir(source_dir) == False:
      print(f"{source_dir} not a valid directory")
      return -1

    i = 0
    #Go through the files inside the source_dir
    for dir in os.listdir(source_dir):
      for file in os.listdir(os.path.join(source_dir, dir)):

        if i not in self.labelDictionary:
          self.labelDictionary[i] = dir

        try:
          Image.open(os.path.join(source_dir, dir, file))
        except:
          continue
      
        self.images.append(os.path.join(source_dir, dir, file))
        self.labels.append(i)
      i += 1

  #Function that returns the size of the dataset
  def __len__(self):
    return len(self.images)

  #Function that returns the ith image and label after converting the image
  def __getitem__(self, i):

    #Opening an Image
    image = Image.open(self.images[i])
  
    label = self.labels[i]

    #All the transformations
    if self.transform is not None:
      image = self.transform(image)

    img = self.toTensor(image)
    if img.shape[0] != 3:
      rgbimg = Image.new("RGB", image.size)
      rgbimg.paste(image)
      img = self.toTensor(rgbimg)

    # image = image.float()
    return img, label
  
  def getLabels(self):
    return self.labels
  
  def getDictionary(self):
    return self.labelDictionary

In [6]:
transform = transforms.Compose([
    transforms.Resize((image_size, image_size))
])

In [7]:
data = CreateDataset(source_dir=data_folder, transform=transform)
train_dataset, valid_dataset, test_dataset = random_split(data, [0.8, 0.1, 0.1])

temp_train = train_dataset[0]
temp_valid = valid_dataset[0]
temp_test = test_dataset[0]

temp = data.getLabels()
# print(data.getDictionary()[0], temp.count(0))

# visualize(temp_train[0], temp_train[1])
# visualize(temp_valid[0], temp_valid[1])
# visualize(temp_test[0], temp_test[1])

c:\Users\utsav\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\Image.py:3218: DecompressionBombWarning: Image size (98130452 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [8]:
phases = {
    'train': train_dataset,
    'valid': valid_dataset,
    'test': test_dataset
}

loader = {
    phase: DataLoader(ds, batch_size=batch_size, shuffle=(phase=='train'))
    for phase, ds in phases.items()
}

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
model = model.to(device)

In [10]:
def evaluate(model, loader, device, criterion, mode='validation'):
    # Set the model to evaluation mode
    model.eval()
  
    # Initialize variables to track total data, correct predictions, and total loss
    total = 0
    total_correct = 0
    total_loss = 0
  
    # Iterate through the data loader
    for i, (images, labels) in enumerate(loader[mode]):

        # Move images and labels to the specified device (CPU or GPU)
        images = images.to(device)
        labels = labels.to(device)
    
        # Disable gradient computation
        with torch.no_grad():

            # Forward pass
            outputs = model(images)
            
            # Calculate the loss
            loss = criterion(outputs, labels)
      
            # Accumulate the total loss
            total_loss += loss.item() * images.size(0)
            
            # Accumulate the total number of data points
            total += images.size(0)
            
            # Calculate the number of correct predictions
            _, predictions = outputs.max(1)
            total_correct += (labels == predictions).sum()
  
    # Calculate average loss and accuracy
    loss = total_loss / total
    accuracy = total_correct / total
    
    # Print evaluation metrics
    print(f'{mode} epoch {epoch}: Loss({loss:6.4f}) Accuracy ({accuracy:6.4f})')


In [11]:
for epoch in range(epochs):
    # Set the model to training mode
    model.train()

    # Initialize variables to track total data, correct predictions, and total loss for this epoch
    total = 0
    total_correct = 0
    total_loss = 0

    # Iterate through the training data loader
    for i, (images, labels) in tqdm(enumerate(loader['train']), total=len(loader['train'])):

        # Move images and labels to the specified device (CPU or GPU)
        images = images.to(device)
        labels = labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        # Accumulate the total loss
        total_loss += loss.item() * images.size(0)
        
        # Accumulate the total number of data points
        total += images.size(0)
        
        # Calculate the number of correct predictions
        _, predictions = outputs.max(1)
        total_correct += (predictions == labels).sum()

    # Calculate average loss and accuracy for this epoch
    accuracy = total_correct / total
    loss = total_loss / total

    # Print training metrics for this epoch
    print(f'Train epoch {epoch}: Loss({loss:6.4f}) Accuracy ({accuracy:6.4f})')

    # Evaluate the model on the validation set
    evaluate(model, loader, device, criterion, mode='valid')


100%|██████████| 189/189 [02:31<00:00,  1.24it/s]


Train epoch 0: Loss(0.7539) Accuracy (0.7161)
valid epoch 0: Loss(0.5134) Accuracy (0.7974)


100%|██████████| 189/189 [02:32<00:00,  1.24it/s]


Train epoch 1: Loss(0.6186) Accuracy (0.7737)
valid epoch 1: Loss(0.4950) Accuracy (0.8066)


100%|██████████| 189/189 [02:31<00:00,  1.25it/s]


...

Train epoch 37: Loss(0.4619) Accuracy (0.8220)
valid epoch 37: Loss(0.4261) Accuracy (0.8424)


100%|██████████| 189/189 [02:28<00:00,  1.27it/s]


Train epoch 38: Loss(0.4718) Accuracy (0.8197)
valid epoch 38: Loss(0.4266) Accuracy (0.8411)


100%|██████████| 189/189 [02:27<00:00,  1.28it/s]


Train epoch 39: Loss(0.4599) Accuracy (0.8246)
valid epoch 39: Loss(0.4338) Accuracy (0.8344)


In [12]:
evaluate(model, loader, device, criterion, mode='test')

test epoch 39: Loss(0.5200) Accuracy (0.8106)


In [13]:
torch.save(model, "2nd-model")

- **Load the Trained Model:** Load the trained VGG16 model along with its weights using `torch.load(path_to_model)` function.


# Load the trained VGG16 model
```
model = torch.load('path_to_your_trained_model.pth')
```



# Transform the images to fit img size
```
from torchvision import transforms
from PIL import Image

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
```
# You only have the run the first 3 cells before you call the model.eval method 

```
 The labels used in training are [drawings,hentai,neutral,porn,sexy]
 ```